## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

In [2]:
# imports for langchain and Chroma and plotly

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [3]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [4]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [7]:

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, 
    with the ability to crawl internal links.
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')

        # Title
        self.title = soup.title.string if soup.title else "No title found"

        # print(self.title)
        # Clean up the body text: remove scripts, styles, images, inputs
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        # Extract all raw links
        raw_links = [link.get('href') for link in soup.find_all('a')]
        # Filter out None or empty
        self.links = [link for link in raw_links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

    def crawl_internal_links(self, visited=None):
        """
        Crawls only internal links (same domain) from this page
        and returns a list of sub-pages' data.
        
        visited: a set to keep track of URLs we've already visited,
                 so we avoid infinite loops.
        """
        if visited is None:
            visited = set()

        results = []
        base_domain = urlparse(self.url).netloc

        for link in self.links:
            # Build an absolute URL (to handle relative links)
            abs_link = urljoin(self.url, link)
            link_domain = urlparse(abs_link).netloc

            # Check if this link is "internal" (same domain) 
            if link_domain == base_domain and abs_link not in visited:
                visited.add(abs_link)
                try:
                    # Instantiate a new Website object for this internal link
                    sub_site = Website(abs_link)

                    # You could store or process sub_site's data here:
                    results.append({
                        "url": abs_link,
                        "title": sub_site.title,
                        "text": sub_site.text,
                        "links": sub_site.links
                    })

                    # (Optional) Recursively crawl further if you want deeper coverage:
                    deeper_links = sub_site.crawl_internal_links(visited=visited)
                    results.extend(deeper_links)

                except Exception as e:
                    print(f"Error scraping {abs_link}: {e}")

        return results

site = Website("https://ashesofcreation.wiki/")
# site = Website("http://Gratco.com")
# site.links
front_dict = {
    "url": site.url,
    "title": site.title,
    "text": site.text,
    "links": site.links
}
site_dict = site.crawl_internal_links()

site_dict.insert(0, front_dict)

# for i, page_info in enumerate(site_dict, start=0):
#     print(f"\nPage #{i} => URL: {page_info['url']}")
#     print("Links found:")
#     for link in page_info["links"]:
#         print(link)

# now we make a list of the text of each page
documents = []
for page_info in site_dict:
    doc = Document(page_content=page_info["text"])
    doc.metadata = {"doc_type": page_info["title"]}
    documents.append(doc)
    


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Please note:

In the next cell, we split the text into chunks.

2 students let me know that the next cell crashed their computer.  
They were able to fix it by changing the chunk_size from 1,000 to 2,000 and the chunk_overlap from 200 to 400.  
This shouldn't be required; but if it happens to you, please make that change!  
(Note that LangChain may give a warning about a chunk being larger than 1,000 - this can be safely ignored).

_With much thanks to Steven W and Nir P for this valuable contribution._

In [8]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [9]:
len(chunks)

9833

In [10]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: Search - Ashes of Creation Wiki, User:Lex - Ashes of Creation Wiki, User contributions for CheriQuentin - Ashes of Creation Wiki, User contributions for Lex - Ashes of Creation Wiki, File:F5 OtneWEAA-4mI.jpg - Ashes of Creation Wiki, User:Maintenance script - Ashes of Creation Wiki, Select language - Ashes of Creation Wiki, Engaging and immersive story - Ashes of Creation Wiki, Recent changes - Ashes of Creation Wiki, User talk:Lex - Ashes of Creation Wiki, Blocked users - Ashes of Creation Wiki, Sand Mother - Ashes of Creation Wiki, File:StevenLeak.png - Ashes of Creation Wiki, User:CheriQuentin - Ashes of Creation Wiki, Tags - Ashes of Creation Wiki, Kickstarter - Ashes of Creation Wiki, Ashes of Creation Wiki, Reactive world - Ashes of Creation Wiki, Internal error - Ashes of Creation Wiki, File:AshesOfCreation.mp4 - Ashes of Creation Wiki, Cataphract Of Valor Gauntlets - Ashes of Creation Wiki, Phoenix initiative - Ashes of Creation Wiki, Ashes of Creation - A

## A sidenote on Embeddings, and "Auto-Encoding LLMs"

We will be mapping each chunk of text into a Vector that represents the meaning of the text, known as an embedding.

OpenAI offers a model to do this, which we will use by calling their API with some LangChain code.

This model is an example of an "Auto-Encoding LLM" which generates an output given a complete input.
It's different to all the other LLMs we've discussed today, which are known as "Auto-Regressive LLMs", and generate future tokens based only on past context.

Another example of an Auto-Encoding LLMs is BERT from Google. In addition to embedding, Auto-encoding LLMs are often used for classification.

### Sidenote

In week 8 we will return to RAG and vector embeddings, and we will use an open-source vector encoder so that the data never leaves our computer - that's an important consideration when building enterprise systems and the data needs to remain internal.

In [13]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk

# embeddings = OpenAIEmbeddings()

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\gratc\AppData\Local\Temp\ipykernel_32864\1687908394.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\gratc\anaconda3\envs\llms\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gratc\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
# # Check if a Chroma Datastore already exists - if so, delete the collection to start from scratch

# if os.path.exists(db_name):
#     Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# load if exists 
selected_db = site.title+db_name
if os.path.exists(selected_db):
    # Load the existing vectorstore from the persist directory
    vectorstore = Chroma(persist_directory=selected_db, embedding_function=embeddings)

    # Print the number of documents in the loaded vectorstore
    print(f"Vectorstore loaded with {vectorstore._collection.count()} documents")
else:
    # Create vectorstore
    vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=selected_db)
    print(f"Vectorstore created with {vectorstore._collection.count()} documents")



Vectorstore loaded with 9833 documents


In [18]:
# Get one vector and find how many dimensions it has

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 384 dimensions


## Visualizing the Vector Store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [19]:
# Prework

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
# colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [21]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)
# Ensure the perplexity is less than the number of samples
perplexity_value = min(30, len(vectors) - 1)

tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity_value)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [22]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42, perplexity=perplexity_value)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color="blue", opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()